In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F


from torchvision.datasets import MNIST
from torchvision import transforms as T

import matplotlib.pyplot as plt
import numpy as np

In [6]:
dataset = MNIST('~/Developer/datasets', transform=T.ToTensor())

In [45]:
def shit_image(image):
    dx = np.random.randint(-3, 4)
    dy = np.random.randint(-3, 4)
    image = torch.roll(image, dx, -1)
    image = torch.roll(image, dy, -2)
    return image, dx, dy

In [ ]:
# Encoder
# Decoder
# train function

In [46]:
class Autoencoder_conv(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            # N, 1, 28, 28
            nn.Conv2d(1, 32, 3, stride=2, padding=1), # N, 32, 14, 14
            nn.ReLU(),
            nn.Conv2d(32, 16, 3, stride=2, padding=1), # N, 16, 7, 7
            nn.ReLU(),
            nn.Conv2d(16, 16, 4, stride=2, padding=1), # N, 16, 3, 3
            nn.ReLU(),
            nn.Conv2d(16, 8, 3), # N, 8, 1, 1
            nn.ReLU(),
            nn.Conv2d(8, 4, 1), # N, 2, 1, 1
        )
        self.decoder = nn.Sequential(
            # N, 64, 1, 1
            nn.ConvTranspose2d(4, 8, 1), # N, 2, 1, 1
            nn.ReLU(),
            nn.ConvTranspose2d(8, 16, 3), # N, 
            nn.ReLU(),
            nn.ConvTranspose2d(16, 16, 4, stride=2, padding=1, output_padding=1), # N, 
            nn.ReLU(),
            nn.ConvTranspose2d(16, 32, 3, stride=2, padding=1, output_padding=1), # N,
            nn.ReLU(),
            nn.ConvTranspose2d(32, 1, 3, stride=2, padding=1, output_padding=1),  # N, 
            nn.Sigmoid()
        )
        
    def encode(self, x):
        return self.encoder(x)
    
    
    def decode(self, z):
        return self.decoder(z)

#     def forward(self, x):
#         encoded = self.encoder(x)
#         decoded = self.decoder(encoded)
#         return decoded

In [ ]:
def train(model, device, train_loader, optimizer, epochs, conv):
    loss = 0

    for batch_features, target in train_loader:
        # reshape mini-batch data to [N, 784] matrix
        # load it to the active device
        
#         if conv == False:
#             batch_features = batch_features.view(-1, 784).to(device)
        new_features, dx, dy = shit_image(batch_features)
        # reset the gradients back to zero
        # PyTorch accumulates gradients on subsequent backward passes
        optimizer.zero_grad()
        
        # compute reconstructions
        # phase 1
        z = model.encode(batch_features)
        # phase 2
        z_shifted = z + torch.Tensor([dx, dy, 0, 0]).view(1, 2, 1, 1).to(device)
        # phase 3
        outputs = model.decode(z_shifted)
#         outputs = model(batch_features)

        # compute training reconstruction loss
        train_loss = criterion(outputs, new_features)
        
        # compute accumulated gradients
        train_loss.backward()
        
        # perform parameter update based on current gradients
        optimizer.step()
        
        # add the mini-batch training loss to epoch loss
        loss += train_loss.item()
    
    # compute the epoch training loss
    loss = loss / len(train_loader)
    
    # display the epoch training loss
    print("epoch : {}/{}, loss = {:.6f}".format(epochs + 1, epochs, loss))
    return outputs,batch_features

In [47]:
# 5 x 5
# [-2, -2] [-2. -1]., [-2. 0]., [-2. +1], [-2, +2]
# [-1, -2] [-2. -1]., [-2. 0]., [-2. +1], [-2, +2]
# [0, -2] [-2. -1]., [-2. 0]., [-2. +1], [-2, +2]
# [1, -2] [-2. -1]., [-2. 0]., [-2. +1], [-2, +2]
# [2, -2] [-2. -1]., [-2. 0]., [-2. +1], [-2, +2]